## Part 1: Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import regularizers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [4]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Age', 'DistanceFromHome', 'Education', 'JobSatisfaction', 'NumCompaniesWorked', 'OverTime', 'TotalWorkingYears', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion']

# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Age                      1470 non-null   int64 
 1   DistanceFromHome         1470 non-null   int64 
 2   Education                1470 non-null   int64 
 3   JobSatisfaction          1470 non-null   int64 
 4   NumCompaniesWorked       1470 non-null   int64 
 5   OverTime                 1470 non-null   object
 6   TotalWorkingYears        1470 non-null   int64 
 7   WorkLifeBalance          1470 non-null   int64 
 8   YearsAtCompany           1470 non-null   int64 
 9   YearsSinceLastPromotion  1470 non-null   int64 
dtypes: int64(9), object(1)
memory usage: 115.0+ KB


In [5]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_df['OverTime'] = X_df['OverTime'].map({'No': 0, 'Yes': 1})

X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   Age                      1470 non-null   int64
 1   DistanceFromHome         1470 non-null   int64
 2   Education                1470 non-null   int64
 3   JobSatisfaction          1470 non-null   int64
 4   NumCompaniesWorked       1470 non-null   int64
 5   OverTime                 1470 non-null   int64
 6   TotalWorkingYears        1470 non-null   int64
 7   WorkLifeBalance          1470 non-null   int64
 8   YearsAtCompany           1470 non-null   int64
 9   YearsSinceLastPromotion  1470 non-null   int64
dtypes: int64(10)
memory usage: 115.0 KB


C:\Users\jason\AppData\Local\Temp\ipykernel_32864\3913651247.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_df['OverTime'] = X_df['OverTime'].map({'No': 0, 'Yes': 1})


In [6]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df)

In [ ]:
# Create a StandardScaler
# Fit the StandardScaler to the training data
scaler = StandardScaler().fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_encoder.fit_transform(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
department_train_encoded = department_encoder.transform(y_train[['Department']])
department_test_encoded = department_encoder.transform(y_test[['Department']])

department_test_encoded

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [9]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoded = attrition_encoder.fit_transform(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
attrition_train_encoded = attrition_encoder.transform(y_train[['Attrition']])
attrition_test_encoded = attrition_encoder.transform(y_test[['Attrition']])

attrition_test_encoded

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.

## Part 2: Create, Compile, and Train the Model

In [10]:
# Find the number of columns in the X training data.
n_features = X_train.shape[1]
print(n_features)

# Create the input layer
input_layer = layers.Input(shape=(n_features,), name='input_features')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu', name='shared_layer1')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu', name='shared_layer2')(shared_layer1)

10


In [13]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden1 = layers.Dense(64, activation='relu', name='department_hidden1')(shared_layer2)
department_hidden2 = layers.Dense(32, activation='relu', name='department_hidden2')(department_hidden1)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', 
    kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01),
      name='department_output')(department_hidden2)

In [14]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden1 = layers.Dense(64, activation='relu', name='attrition_hidden1')(shared_layer2)
attrition_hidden2 = layers.Dense(32, activation='relu', name='attrition_hidden2')(attrition_hidden1)
# Create the output layer
attrition_output = layers.Dense(2, activation='sigmoid', kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01), name='attrition_output')(attrition_hidden2)

In [15]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
               loss={'department_output' : 'categorical_crossentropy', 'attrition_output' : 'binary_crossentropy'},
                 metrics={'department_output' : 'accuracy', 'attrition_output' : 'accuracy'})

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_features      │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer1       │ (None, 64)        │        704 │ input_features[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer2       │ (None, 32)        │      2,080 │ shared_layer1[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_hidden1  │ (None, 64)        │      2,112 │ shared_layer2[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_hidden1   │ (None, 64)        │      2,112 │ shared_layer2[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_hidden2  │ (None, 32)        │      2,080 │ department_hidde… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_hidden2   │ (None, 32)        │      2,080 │ attrition_hidden… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ department_hidde… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ attrition_hidden… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 11,333 (44.27 KB)

 Trainable params: 11,333 (44.27 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Train the model
training = model.fit(
    X_train,
    {
        'department_output': department_train_encoded,
        'attrition_output': attrition_train_encoded
    },
    epochs = 100,
    batch_size = 32,
)

Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - attrition_output_accuracy: 0.8194 - attrition_output_loss: 0.5505 - department_output_accuracy: 0.5618 - department_output_loss: 0.9319 - loss: 1.9146
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8179 - attrition_output_loss: 0.4381 - department_output_accuracy: 0.6561 - department_output_loss: 0.8012 - loss: 1.6294
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8179 - attrition_output_loss: 0.4401 - department_output_accuracy: 0.6274 - department_output_loss: 0.7997 - loss: 1.5913
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - attrition_output_accuracy: 0.8459 - attrition_output_loss: 0.3999 - department_output_accuracy: 0.6630 - department_output_loss: 0.7629 - loss: 1.4795
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8387 - attrition_output_loss: 0.4131 - department_output_accuracy: 0.6754 - department_output_loss: 0.

In [17]:
# Evaluate the model with the testing data
results = model.evaluate(np.array(X_test),
                         {
                             'department_output': department_test_encoded,
                             'attrition_output': attrition_test_encoded
                         },)

results

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - attrition_output_accuracy: 0.8457 - attrition_output_loss: 0.3611 - department_output_accuracy: 0.5920 - department_output_loss: 0.9597 - loss: 1.3553  


[1.3214447498321533,
 0.9177563190460205,
 0.3668093681335449,
 0.8505434989929199,
 0.6005434989929199]

In [18]:
# Print the accuracy for both department and attrition
print("Attrition Accuracy: %.2f%%" % (results[3] * 100))
print("Department Accuracy: %.2f%%" % (results[4] * 100))

Attrition Accuracy: 85.05%
Department Accuracy: 60.05%


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Based on the results, I do not think accuracy is the best metric to use on this data.  Looking at the results, the attrition output had and accuracy around 85%, while the department output was around 60%.  This difference suggest that the data is imbalanced.  Metrics like precision or AUC-ROC could provide model performance.

2. I used softmax for the department output layer and sigmoid for the attrition output layer.

    Softmax was used for the department output layer because of the data being multi-class classification. Softmax was used because a probability distribution across all three departments was needed, and Softmax would ensure that the probabilities would sum to 1. Thus allowing the model to predict the most likely department for each employee.

    Sigmoid was used for the attrition output layer because of the data being binary.  Sigmoid is better at dealing with binary data sets than Softmax and outputs a single value between 0 and 1.

3. The model could be improved by:
    A. techniques like oversampling or SMOTE could help with the attrition data by creating additional samples to boost the data.
    
    B. using diffrent sets of features or adding new features may help.

    C. hyperparameter tuning the model (ammount of layers, number of nodes, number of epochs) could help.